In [1]:
#importing all of our libraries and data
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.applications import VGG16, InceptionResNetV2, ResNet50, Xception

In [2]:
#passing the path of the directory containing training and testing data
train_dir = "../input/fer2013/train"
test_dir = "../input/fer2013/test"

In [3]:
IMG_SIZE = 48 #defining the size of our image

In [4]:
#performing data augmentation on our data
train_datagen = image.ImageDataGenerator(#rotation_range = 180,
                                         width_shift_range = 0.1,
                                         height_shift_range = 0.1,
                                         #brightness_range = [0.1,1.1],
                                         horizontal_flip = True,
                                         #vertical_flip = True,
                                         rescale = 1./255,
                                         zoom_range = 0.2,
                                         validation_split = 0.2
                                        )
validation_datagen = image.ImageDataGenerator(rescale = 1./255,
                                             validation_split = 0.2)

In [7]:
#taking the path of the directory and generating batches of our augmented data
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                               target_size = (IMG_SIZE,IMG_SIZE),
                                                               color_mode = "grayscale",
                                                               class_mode = "categorical",
                                                               batch_size = 64,
                                                               #shuffle = False,
                                                               subset = "training"
)

validation_generator = validation_datagen.flow_from_directory(directory = test_dir,
                                                             target_size = (IMG_SIZE,IMG_SIZE),
                                                             color_mode = "grayscale",
                                                             class_mode = "categorical",
                                                             batch_size = 64,
                                                             #shuffle = True,
                                                             subset = "validation"
)

Found 22968 images belonging to 7 classes.
Found 1432 images belonging to 7 classes.


In [36]:
#model = tf.keras.Sequential()
#model.add(VGG16(include_top = False,weights = '../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',input_shape= (IMG_SIZE,IMG_SIZE,3)))
#model.add(MaxPool2D(padding = 'same'))
#model.add(BatchNormalization(axis=-1))
#model.add(Flatten())
#model.add(Dense(7,activation = 'softmax'))

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',input_shape = (IMG_SIZE,IMG_SIZE,1),padding = 'same'),
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'same'),
    tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Conv2D(128,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(128,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Conv2D(256,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(256,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(256,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Conv2D(512,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(512,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(512,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Conv2D(512,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(512,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(512,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096,activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096,activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(7,activation = 'softmax')
])

In [10]:
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
optimizer = Adam(lr = 0.0001)
model.compile(loss = "categorical_crossentropy",optimizer = optimizer,metrics = ['accuracy'])

In [11]:
epochs = 30
batch_size = 64

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 128)       0

In [36]:
history = model.fit(train_generator, steps_per_epoch =22968//64, epochs = epochs,validation_data = validation_generator,validation_steps = 1432//64)

Epoch 1/30
358/358 [==============================] - 35s 95ms/step - loss: 1.8927 - accuracy: 0.2447 - val_loss: 1.8244 - val_accuracy: 0.2464
Epoch 2/30
358/358 [==============================] - 33s 93ms/step - loss: 1.8496 - accuracy: 0.2618 - val_loss: 1.8183 - val_accuracy: 0.2450
Epoch 3/30
358/358 [==============================] - 33s 93ms/step - loss: 1.8191 - accuracy: 0.2466 - val_loss: 1.8149 - val_accuracy: 0.2486
Epoch 4/30
358/358 [==============================] - 33s 92ms/step - loss: 1.8278 - accuracy: 0.2489 - val_loss: 1.8232 - val_accuracy: 0.2479
Epoch 5/30
358/358 [==============================] - 34s 94ms/step - loss: 1.8315 - accuracy: 0.2143 - val_loss: 1.8146 - val_accuracy: 0.2479
Epoch 6/30
358/358 [==============================] - 33s 93ms/step - loss: 1.8235 - accuracy: 0.2390 - val_loss: 1.8163 - val_accuracy: 0.2464
Epoch 7/30
358/358 [==============================] - 34s 95ms/step - loss: 1.7868 - accuracy: 0.2831 - val_loss: 1.8124 - val_accuracy: